In [ ]:
# NB: This is for spark running on parquet files converted from
#     baconbits skims. This is a prototype, there is lots of boilerplate.
#     We're making it better :-)

import pyspark.sql
import os
import sys
from pyarrow.compat import guid
from coffea.processor.spark.detail import _spark_initialize

# The following line is necessary because we're working in a
# virtualenv. Without it, executors will use the wrong interpreter!
#os.environ['PYSPARK_PYTHON'] = sys.executable

#    .config("spark.driver.extraClassPath","/home/cms.lgray/sparkMeasure/target/scala-2.12/spark-measure_2.12-0.14-SNAPSHOT.jar") \
#    .master('local[*]') \

conf = pyspark.sql.SparkSession.builder \
    .appName('baconbits-spark-%s' % guid()) \
    .master('spark://cmsspark-submit.fnal.gov:7077') \
    .config('spark.executor.memory', "20g") \
    .config('spark.executor.cores', "4") \
    .config('spark.sql.execution.arrow.enabled','true') \
    .config("spark.blockManager.port", "8000") \
    .config('spark.sql.execution.arrow.pyspark.enabled','true') \
    .config('spark.sql.execution.arrow.maxRecordsPerBatch', 200000) \
    .config('spark.driver.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar') \
    .config('spark.executor.extraClassPath','file:///opt/hadoop-xrootd/hadoop-xrootd-1.0.4-jar-with-dependencies.jar')
session = _spark_initialize(config=conf,log_level='WARN',laurelin_version='0.3.0')
sc = session.sparkContext
sc.setLogLevel("WARN")
spark = session

#from sparkmeasure import TaskMetrics
#taskmetrics = TaskMetrics(spark)

partitionsize = 200000
thread_workers = 16


In [10]:
import pyspark.sql.functions as fn
from tqdm import tqdm
import json

datasets = {}

with open('metadata/samplefiles.json') as f:
    temp = json.load(f)
    for dsgroup,datasetlist in temp.items():
        if dsgroup != 'Hbb_2017': continue
        datasets = datasetlist

datasets_spark = {}

for ds, flist in datasets.items():
    if '/skim/' in flist[0]:
        datasets_spark[ds] = {'files': flist, 'treename': 'otree'}
    else:
        datasets_spark[ds] = {'files': flist, 'treename': 'Events'}

print(datasets_spark)

{'GluGluHToBB_M-125_13TeV_powheg_MINLO_NNLOPS_pythia8': {'files': ['root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.02/skim/GluGluHToBB_M_125_13TeV_powheg_MINLO_NNLOPS_pythia8_0.root'], 'treename': 'otree'}, 'VBFHToBB_M_125_13TeV_powheg_pythia8_weightfix': {'files': ['root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.01/skim/VBFHToBB_M_125_13TeV_powheg_pythia8_weightfix_0.root'], 'treename': 'otree'}, 'TTToSemiLeptonic_TuneCP5_13TeV_powheg_pythia8': {'files': ['root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.01/skim/TTToSemiLeptonic_TuneCP5_13TeV_powheg_pythia8_0.root', 'root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.01/skim/TTToSemiLeptonic_TuneCP5_13TeV_powheg_pythia8_1.root', 'root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle/zprimebits-v15.01/skim/TTToSemiLeptonic_TuneCP5_13TeV_powheg_pythia8_2.root', 'root://cmseos.fnal.gov//eos/uscms/store/user/lpcbacon/dazsle

In [ ]:
#get the hbb analysis worker
from coffea.util import load

processor_instance = load('boostedHbbProcessor.coffea')

In [ ]:
import time
from coffea.processor import run_spark_job
from coffea.processor.spark.spark_executor import spark_executor

tic = time.time()
final_accumulator = run_spark_job(datasets_spark, processor_instance, spark_executor, 
                                  spark=spark, partitionsize=partitionsize, thread_workers=thread_workers,
                                  executor_args={'file_type': 'root'})
dt = time.time() - tic


loading:   0%|          | 0/52 [00:00<?, ?datasets/s]

pyspark version: 2.4.3


loading:   0%|          | 0/52 [00:02<?, ?datasets/s]


Py4JJavaError: An error occurred while calling o125.load.
: java.util.NoSuchElementException: Could not load "Events"
	at edu.vanderbilt.accre.laurelin.root_proxy.TFile.getProxy(TFile.java:135)
	at edu.vanderbilt.accre.laurelin.Root$TTreeDataSourceV2Reader.<init>(Root.java:219)
	at edu.vanderbilt.accre.laurelin.Root.createReader(Root.java:462)
	at edu.vanderbilt.accre.laurelin.Root.createReader(Root.java:444)
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2Relation$SourceHelpers.createReader(DataSourceV2Relation.scala:155)
	at org.apache.spark.sql.execution.datasources.v2.DataSourceV2Relation$.create(DataSourceV2Relation.scala:172)
	at org.apache.spark.sql.DataFrameReader.load(DataFrameReader.scala:204)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(M

In [ ]:
nevt = sum(spark_executor.counts.values())
print('processed:',nevt,'events')
print('total time: ',dt/60)
print('μs/evt', dt/nevt*1e6)
print('Mevt/s', nevt/dt/1e6)


In [ ]:
from coffea import hist
import gzip
import pickle
import numexpr
import numpy as np

nbins = sum(sum(arr.size for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
nfilled = sum(sum(np.sum(arr>0) for arr in h._sumw.values()) for h in final_accumulator.values() if isinstance(h, hist.Hist))
print("Processed %.1fM events" % (nevt/1e6, ))
print("Filled %.1fM bins" % (nbins/1e6, ))
print("Nonzero bins: %.1f%%" % (100*nfilled/nbins, ))

# Pickle is not very fast or memory efficient, will be replaced by something better soon
with lz4f.open("hists.cpkl.lz4", mode="wb", compression_level=6) as fout:
    cpkl.dump(final_accumulator, fout)

#dt = time.time() - tstart
#print("%.2f us*cpu/event overall" % (1e6*dt*nworkers/final_accumulators['nentries'], ))


In [5]:
spark.stop()
